In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [55]:
import pandas as pd
import numpy as np
from fastai.vision import *

In [23]:
path = Path('data_faces/Faces/')


In [24]:
def validation_func(x):
    return 'valid' in x

In [25]:
tfms = get_transforms(do_flip=False, flip_vert=False, max_rotate=30, max_lighting=0.3)

In [27]:
src = (ImageList.from_csv(path, csv_name='labels.csv')
       .split_by_valid_func(validation_func)
       .label_from_df(cols='label',label_delim=' '))

data = (src.transform(tfms, size=128)
       .databunch(bs=128).normalize(imagenet_stats))

In [52]:
acc_02 = partial(accuracy_thresh, thresh=0.2)
acc_03 = partial(accuracy_thresh, thresh=0.3)
acc_04 = partial(accuracy_thresh, thresh=0.4)
acc_05 = partial(accuracy_thresh, thresh=0.5)
f_score = partial(fbeta, thresh=0.2)
learner = cnn_learner(data,models.resnet50,metrics=[acc_02, acc_03, acc_04, acc_05, f_score])

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [56]:
learner.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy_thresh,accuracy_thresh,accuracy_thresh,accuracy_thresh,fbeta,time
0,0.236337,0.223498,0.855414,0.885176,0.897613,0.900135,0.814291,13:47


In [57]:
learner.export('facial_features.pkl')